In [1]:
!pip install mlxtend

import pandas as pd
import matplotlib.pyplot as plt
from apyori import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
import pandas as pd

file_path = "Project datasets/"
file_name = "D1.csv"
df = pd.read_csv(f"{file_path}{file_name}",index_col='patient_id')
df

,global_num,date,location,latitude,longitude
patient_id,,,,,
1000000001,2.0,22/01/2020,Gyeonggi-do_Gimpo-si,37.615246,126.715632
1000000001,2.0,24/01/2020,Seoul_Jung-gu,37.567241,127.005659
1000000002,5.0,26/01/2020,Seoul_Seongdong-gu,37.563992,127.029534
1000000002,5.0,27/01/2020,Seoul_Dongdaemun-gu,37.566262,127.065815
1000000002,5.0,28/01/2020,Seoul_Gangnam-gu,37.523674,127.046543
...,...,...,...,...,...
6100000083,NaN,6/03/2020,Daegu_Buk-gu,35.891794,128.588890
6100000085,NaN,16/03/2020,Gyeongsangnam-do_Changwon-si,35.227956,128.685595
6100000086,NaN,14/03/2020,Daegu_Dalseong-gun,35.857185,128.466686


In [3]:
df["date"] = pd.to_datetime(df["date"], format='%d/%m/%Y')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1509 entries, 1000000001 to 6100000090
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   global_num  959 non-null    float64       
 1   date        1509 non-null   datetime64[ns]
 2   location    1509 non-null   object        
 3   latitude    1509 non-null   float64       
 4   longitude   1509 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 70.7+ KB


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1509 entries, 1000000001 to 6100000090
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   global_num  959 non-null    float64       
 1   date        1509 non-null   datetime64[ns]
 2   location    1509 non-null   object        
 3   latitude    1509 non-null   float64       
 4   longitude   1509 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 70.7+ KB


In [5]:
df.head(10)

,global_num,date,location,latitude,longitude
patient_id,,,,,
1000000001,2.0,2020-01-22,Gyeonggi-do_Gimpo-si,37.615246,126.715632
1000000001,2.0,2020-01-24,Seoul_Jung-gu,37.567241,127.005659
1000000002,5.0,2020-01-26,Seoul_Seongdong-gu,37.563992,127.029534
1000000002,5.0,2020-01-27,Seoul_Dongdaemun-gu,37.566262,127.065815
1000000002,5.0,2020-01-28,Seoul_Gangnam-gu,37.523674,127.046543
1000000004,7.0,2020-01-30,Seoul_Jungnang-gu,37.612772,127.098167
1000000005,9.0,2020-01-31,Seoul_Jungnang-gu,37.612772,127.098167
1000000006,10.0,2020-01-30,Gyeonggi-do_Goyang-si,37.641141,126.791968
1000000007,11.0,2020-01-30,Gyeonggi-do_Goyang-si,37.641141,126.791968


In [6]:
print(df.duplicated())

patient_id
1000000001    False
1000000001    False
1000000002    False
1000000002    False
1000000002    False
              ...  
6100000083     True
6100000085    False
6100000086    False
6100000090    False
6100000090     True
Length: 1509, dtype: bool


In [7]:
unique_locations = df['location'].nunique()
unique_locations

151

In [8]:
unique_date = df['date'].nunique()
unique_date

95

In [9]:
# Dropping the 'global_num' column
df.drop(['global_num','latitude','longitude'],inplace=True,axis=1)

# Print the DataFrame to verify if the column is dropped
df

,date,location
patient_id,,
1000000001,2020-01-22,Gyeonggi-do_Gimpo-si
1000000001,2020-01-24,Seoul_Jung-gu
1000000002,2020-01-26,Seoul_Seongdong-gu
1000000002,2020-01-27,Seoul_Dongdaemun-gu
1000000002,2020-01-28,Seoul_Gangnam-gu
...,...,...
6100000083,2020-03-06,Daegu_Buk-gu
6100000085,2020-03-16,Gyeongsangnam-do_Changwon-si
6100000086,2020-03-14,Daegu_Dalseong-gun


In [10]:
# group by patient id, then list all locations
transactions = df.groupby(['patient_id'])['location'].apply(list)
print(transactions)

patient_id
1000000001                [Gyeonggi-do_Gimpo-si, Seoul_Jung-gu]
1000000002    [Seoul_Seongdong-gu, Seoul_Dongdaemun-gu, Seou...
1000000004                                  [Seoul_Jungnang-gu]
1000000005                                  [Seoul_Jungnang-gu]
1000000006                              [Gyeonggi-do_Goyang-si]
                                    ...                        
6100000082                      [Gyeongsangnam-do_Geochang-gun]
6100000083    [Daegu_Jung-gu, Gyeongsangnam-do_Geoje-si, Dae...
6100000085                       [Gyeongsangnam-do_Changwon-si]
6100000086                                 [Daegu_Dalseong-gun]
6100000090                  [Incheon_Jung-gu, Busan_Gangseo-gu]
Name: location, Length: 891, dtype: object


In [11]:
# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.01))

# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'Busan_Busanjin-gu'}), support=0.012345679012345678, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Busanjin-gu'}), confidence=0.012345679012345678, lift=1.0)]), RelationRecord(items=frozenset({'Busan_Dong-gu'}), support=0.01122334455667789, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Dong-gu'}), confidence=0.01122334455667789, lift=1.0)]), RelationRecord(items=frozenset({'Busan_Dongnae-gu'}), support=0.019079685746352413, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Dongnae-gu'}), confidence=0.019079685746352413, lift=1.0)]), RelationRecord(items=frozenset({'Busan_Haeundae-gu'}), support=0.015712682379349047, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Haeundae-gu'}), confidence=0.015712682379349047, lift=1.0)]), RelationRecord(items=frozenset({'Busan_Seo-gu'}), support=0.0168350

In [12]:
def convert_apriori_results_to_pandas_df(results):
    rules = []

    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                          rule_set.support, rule.confidence, rule.lift])
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support',
                                         'Confidence', 'Lift'])

result_df = convert_apriori_results_to_pandas_df(results)
print(result_df.head(20))

# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(5))

   Left_side                     Right_side   Support  Confidence  Lift
0                         Busan_Busanjin-gu  0.012346    0.012346   1.0
1                             Busan_Dong-gu  0.011223    0.011223   1.0
2                          Busan_Dongnae-gu  0.019080    0.019080   1.0
3                         Busan_Haeundae-gu  0.015713    0.015713   1.0
4                              Busan_Seo-gu  0.016835    0.016835   1.0
5                           Busan_Yeonje-gu  0.057239    0.057239   1.0
6                 Chungcheongnam-do_Asan-si  0.015713    0.015713   1.0
7              Chungcheongnam-do_Cheonan-si  0.030303    0.030303   1.0
8                              Daegu_Buk-gu  0.038159    0.038159   1.0
9                             Daegu_Dong-gu  0.021324    0.021324   1.0
10                            Daegu_Jung-gu  0.054994    0.054994   1.0
11                             Daegu_Nam-gu  0.010101    0.010101   1.0
12                             Daegu_Seo-gu  0.010101    0.01010

In [13]:
# filter rows where location is "Busan_Buk-gu"
busan = df["location"] == "Busan_Buk-gu"
filtered = df.loc[busan]

# select corresponding rows in original dataframe
Busan_IDS = df.loc[filtered.index]

# display Busan_IDS dataframe 
Busan_IDS

,date,location
patient_id,,
1100000105,2020-03-17,Gyeonggi-do_Gimpo-si
1100000105,2020-03-17,Busan_Gangseo-gu
1100000105,2020-03-21,Busan_Buk-gu
1100000105,2020-03-22,Busan_Yeonje-gu
1100000108,2020-03-19,Seoul_Gangseo-gu
1100000108,2020-03-19,Busan_Gangseo-gu
1100000108,2020-03-21,Busan_Buk-gu
1100000108,2020-03-23,Gwangju_Buk-gu
1100000108,2020-03-24,Busan_Yeonje-gu


In [14]:
Busan_routes = Busan_IDS.groupby('patient_id')['location'].apply(list)
Busan_df = pd.DataFrame(Busan_routes)
Busan_df

,location
patient_id,
1100000105,"[Gyeonggi-do_Gimpo-si, Busan_Gangseo-gu, Busan..."
1100000108,"[Seoul_Gangseo-gu, Busan_Gangseo-gu, Busan_Buk..."
1100000129,"[Busan_Dongnae-gu, Busan_Buk-gu]"
1100000131,"[Gwangju_Buk-gu, Busan_Buk-gu, Busan_Yeonje-gu]"
6100000015,"[Busan_Gangseo-gu, Busan_Buk-gu, Gyeongsangnam..."


In [20]:
# type cast the routes from pandas into normal list format and run apriori
Busan_list = list(Busan_routes)
busan_results = list(apriori(Busan_list, min_support=0.01, min_confidence=0.1))

# print first 4 rules
busan_results

[RelationRecord(items=frozenset({'Busan_Buk-gu'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Buk-gu'}), confidence=1.0, lift=1.0)]),
 RelationRecord(items=frozenset({'Busan_Dongnae-gu'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Dongnae-gu'}), confidence=0.2, lift=1.0)]),
 RelationRecord(items=frozenset({'Busan_Gangseo-gu'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Gangseo-gu'}), confidence=0.6, lift=1.0)]),
 RelationRecord(items=frozenset({'Busan_Yeonje-gu'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Busan_Yeonje-gu'}), confidence=0.6, lift=1.0)]),
 RelationRecord(items=frozenset({'Gwangju_Buk-gu'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Gwangju_Buk-gu'}), confidence=0.4, lift=1.0)]),
 Relation

In [21]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift']) 

#apriori results:
result_df = convert_apriori_results_to_pandas_df(busan_results)

#sort by Lift values
result_df = result_df.sort_values(by='Lift', ascending=False)
result_df

,Left_side,Right_side,Support,Confidence,Lift
253,"Gyeongsangnam-do_Uiryeong-gun,Busan_Gangseo-gu","Busan_Buk-gu,Gyeongsangnam-do_Yangsan-si",0.2,1.000000,5.000000
234,Seoul_Gangseo-gu,"Busan_Buk-gu,Gwangju_Buk-gu,Busan_Gangseo-gu",0.2,1.000000,5.000000
173,Gyeongsangnam-do_Uiryeong-gun,"Gyeongsangnam-do_Yangsan-si,Busan_Gangseo-gu",0.2,1.000000,5.000000
251,"Busan_Buk-gu,Gyeongsangnam-do_Uiryeong-gun","Gyeongsangnam-do_Yangsan-si,Busan_Gangseo-gu",0.2,1.000000,5.000000
295,Seoul_Gangseo-gu,"Busan_Buk-gu,Busan_Yeonje-gu,Gwangju_Buk-gu,Bu...",0.2,1.000000,5.000000
...,...,...,...,...,...
78,"Busan_Buk-gu,Gwangju_Buk-gu",Busan_Gangseo-gu,0.2,0.500000,0.833333
37,Busan_Gangseo-gu,Gwangju_Buk-gu,0.2,0.333333,0.833333
38,Gwangju_Buk-gu,Busan_Gangseo-gu,0.2,0.500000,0.833333
75,Busan_Gangseo-gu,"Busan_Buk-gu,Gwangju_Buk-gu",0.2,0.333333,0.833333


In [22]:
onlyfrom_busan_df = result_df[(result_df['Left_side'] == 'Busan_Buk-gu')]
onlyfrom_busan_df.head(4)

,Left_side,Right_side,Support,Confidence,Lift
261,Busan_Buk-gu,"Busan_Yeonje-gu,Gwangju_Buk-gu,Seoul_Gangseo-gu",0.2,0.2,1.0
231,Busan_Buk-gu,"Busan_Gangseo-gu,Gwangju_Buk-gu,Seoul_Gangseo-gu",0.2,0.2,1.0
291,Busan_Buk-gu,"Seoul_Gangseo-gu,Busan_Yeonje-gu,Gwangju_Buk-g...",0.2,0.2,1.0
246,Busan_Buk-gu,"Gyeongsangnam-do_Uiryeong-gun,Gyeongsangnam-do...",0.2,0.2,1.0


In [23]:
filtered_df = result_df[result_df['Left_side'].str.contains('Busan_Buk-gu')]
filtered_df.head(4)

,Left_side,Right_side,Support,Confidence,Lift
251,"Busan_Buk-gu,Gyeongsangnam-do_Uiryeong-gun","Gyeongsangnam-do_Yangsan-si,Busan_Gangseo-gu",0.2,1.0,5.0
237,"Busan_Buk-gu,Seoul_Gangseo-gu","Gwangju_Buk-gu,Busan_Gangseo-gu",0.2,1.0,5.0
141,"Busan_Buk-gu,Gyeongsangnam-do_Yangsan-si",Gyeongsangnam-do_Uiryeong-gun,0.2,1.0,5.0
140,"Busan_Buk-gu,Gyeongsangnam-do_Uiryeong-gun",Gyeongsangnam-do_Yangsan-si,0.2,1.0,5.0


In [24]:
# group Busan_IDS by id column
groups = Busan_IDS.groupby("patient_id")

# create empty routes dataframe
routes = pd.DataFrame(columns=['patient_id', 'start_date', 'start_location', 'end_date', 'end_location'])

for i in range(len(Busan_IDS) - 1):
    if Busan_IDS['location'].iloc[i] == 'Busan_Buk-gu':
        curr_id = Busan_IDS.index[i]
        next_id = Busan_IDS.index[i+1]
        curr_date = Busan_IDS['date'].iloc[i]
        next_date = Busan_IDS['date'].iloc[i+1]
        
        if curr_id == next_id and curr_date <= next_date:
            start_date = curr_date.strftime('%d/%m/%Y')
            end_date = next_date.strftime('%d/%m/%Y')
            start_loc = Busan_IDS['location'].iloc[i]
            end_loc = Busan_IDS['location'].iloc[i+1]
            routes = routes.append({'patient_id': curr_id,
                                    'start_date': start_date,
                                    'start_location': start_loc,
                                    'end_date': end_date,
                                    'end_location': end_loc}, ignore_index=True)
routes

/var/folders/rz/f0plsqsn1f505rm307cbs5yc0000gn/T/ipykernel_35803/4182304662.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  routes = routes.append({'patient_id': curr_id,
/var/folders/rz/f0plsqsn1f505rm307cbs5yc0000gn/T/ipykernel_35803/4182304662.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  routes = routes.append({'patient_id': curr_id,
/var/folders/rz/f0plsqsn1f505rm307cbs5yc0000gn/T/ipykernel_35803/4182304662.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  routes = routes.append({'patient_id': curr_id,
/var/folders/rz/f0plsqsn1f505rm307cbs5yc0000gn/T/ipykernel_35803/4182304662.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

,patient_id,start_date,start_location,end_date,end_location
0,1100000105,21/03/2020,Busan_Buk-gu,22/03/2020,Busan_Yeonje-gu
1,1100000108,21/03/2020,Busan_Buk-gu,23/03/2020,Gwangju_Buk-gu
2,1100000131,21/04/2020,Busan_Buk-gu,21/04/2020,Busan_Yeonje-gu
3,6100000015,21/02/2020,Busan_Buk-gu,23/02/2020,Gyeongsangnam-do_Yangsan-si
